Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

L2 regularization for logistic:

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  tf_beta = tf.placeholder(tf.float32)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + tf_beta * tf.nn.l2_loss(weights))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

results = []
for beta in np.logspace(-4, -2, 10):
    with tf.Session(graph=graph) as session:
      # This is a one-time operation which ensures the parameters get initialized as
      # we described in the graph: random weights for the matrix, zeros for the
      # biases. 
      tf.global_variables_initializer().run()
      print('Initialized')
      for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        feed_dict = {tf_beta: beta}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
          print('Loss at step %d: %f' % (step, l))
          print('Training accuracy: %.1f%%' % accuracy(
            predictions, train_labels[:train_subset, :]))
          # Calling .eval() on valid_prediction is basically like calling run(), but
          # just to get that one numpy array. Note that it recomputes all its graph
          # dependencies.
          print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
          print("")
      message = 'Test beta:%f accuracy: %.1f%%' % (beta, accuracy(test_prediction.eval(), test_labels))
      print(message)
      results.append(message)

[print(message) for message in results]

Initialized
Loss at step 0: 20.874182
Training accuracy: 8.8%
Validation accuracy: 10.9%

Loss at step 100: 2.607229
Training accuracy: 71.6%
Validation accuracy: 70.1%

Loss at step 200: 2.110166
Training accuracy: 74.8%
Validation accuracy: 72.9%

Loss at step 300: 1.843672
Training accuracy: 76.2%
Validation accuracy: 73.7%

Loss at step 400: 1.663778
Training accuracy: 77.2%
Validation accuracy: 74.3%

Loss at step 500: 1.528905
Training accuracy: 77.8%
Validation accuracy: 74.5%

Loss at step 600: 1.421922
Training accuracy: 78.6%
Validation accuracy: 74.8%

Loss at step 700: 1.334296
Training accuracy: 79.1%
Validation accuracy: 75.1%

Loss at step 800: 1.260995
Training accuracy: 79.7%
Validation accuracy: 75.2%

Test beta:0.000100 accuracy: 83.0%
Initialized
Loss at step 0: 19.175467
Training accuracy: 7.8%
Validation accuracy: 10.5%

Loss at step 100: 2.744344
Training accuracy: 71.7%
Validation accuracy: 70.0%

Loss at step 200: 2.256728
Training accuracy: 75.4%
Validation ac

[None, None, None, None, None, None, None, None, None, None]

L2 Reg for NN

In [68]:
batch_size = 128

hidden_nodes = 1024
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  beta_para = tf.placeholder(tf.float32)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes], stddev=0.1))
  biases = tf.Variable(tf.zeros([hidden_nodes]))
    
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
    
  relu_result = tf.nn.relu(logits)

  weights_hiden1 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels], stddev=0.1))
  biases_hiden1 = tf.Variable(tf.zeros([num_labels]))

  logits = tf.matmul(relu_result, weights_hiden1) + biases_hiden1

  l2_reg = tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights_hiden1) + tf.nn.l2_loss(biases) + tf.nn.l2_loss(biases_hiden1)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta_para*l2_reg
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_logits = tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) +biases), weights_hiden1) + biases_hiden1
  valid_prediction = tf.nn.softmax(valid_logits)

  test_logits = tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) +biases), weights_hiden1) + biases_hiden1
  test_prediction = tf.nn.softmax(test_logits)

In [70]:
num_steps = 3001

results = []
for beta_value in np.logspace(-4, -2, 20):
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_para: beta_value}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
          print("")
      message = "Test accuracy when beta is %f: %.1f%%" % (beta_value, accuracy(test_prediction.eval(), test_labels))
      print(message)
      results.append(message)

[print(message) for message in results]    

Initialized
Minibatch loss at step 0: 4.448092
Minibatch accuracy: 4.7%
Validation accuracy: 25.0%

Minibatch loss at step 500: 0.953260
Minibatch accuracy: 82.8%
Validation accuracy: 85.3%

Minibatch loss at step 1000: 0.548219
Minibatch accuracy: 93.8%
Validation accuracy: 86.8%

Minibatch loss at step 1500: 0.492721
Minibatch accuracy: 95.3%
Validation accuracy: 88.1%

Minibatch loss at step 2000: 0.728383
Minibatch accuracy: 88.3%
Validation accuracy: 88.0%

Minibatch loss at step 2500: 0.735832
Minibatch accuracy: 89.1%
Validation accuracy: 88.4%

Minibatch loss at step 3000: 0.655231
Minibatch accuracy: 91.4%
Validation accuracy: 88.3%

Test accuracy when beta is 0.000100: 94.1%
Initialized
Minibatch loss at step 0: 4.992170
Minibatch accuracy: 5.5%
Validation accuracy: 34.5%

Minibatch loss at step 500: 1.022261
Minibatch accuracy: 82.8%
Validation accuracy: 85.6%

Minibatch loss at step 1000: 0.629402
Minibatch accuracy: 93.0%
Validation accuracy: 86.7%

Minibatch loss at step 

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [77]:
batch_size = 128

hidden_nodes = 1024
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  beta_para = tf.placeholder(tf.float32)
  tf_keep_prob = tf.placeholder(tf.float32)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes], stddev=0.1))
  biases = tf.Variable(tf.zeros([hidden_nodes]))
    
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
    
  relu_result = tf.nn.relu(logits)

  relu_result = tf.nn.dropout(relu_result, keep_prob=tf_keep_prob)

  weights_hiden1 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels], stddev=0.1))
  biases_hiden1 = tf.Variable(tf.zeros([num_labels]))

  logits = tf.matmul(relu_result, weights_hiden1) + biases_hiden1

  l2_reg = tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights_hiden1) + tf.nn.l2_loss(biases) + tf.nn.l2_loss(biases_hiden1)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta_para*l2_reg
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_logits = tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) +biases), weights_hiden1) + biases_hiden1
  valid_prediction = tf.nn.softmax(valid_logits)

  test_logits = tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) +biases), weights_hiden1) + biases_hiden1
  test_prediction = tf.nn.softmax(test_logits)

In [78]:
num_steps = 3001
beta_value = 0.000127
results = []

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_para: beta_value, tf_keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("")
   
  session.run(test_prediction, feed_dict={tf_keep_prob: 1})
  message = "Test accuracy when beta is %f: %.1f%%" % (beta_value, accuracy(test_prediction.eval(), test_labels))
  print(message)
  results.append(message)

[print(message) for message in results]    

Initialized
Minibatch loss at step 0: 5.701058
Minibatch accuracy: 10.2%
Validation accuracy: 21.2%

Minibatch loss at step 500: 1.245964
Minibatch accuracy: 81.2%
Validation accuracy: 84.2%

Minibatch loss at step 1000: 0.689382
Minibatch accuracy: 91.4%
Validation accuracy: 85.9%

Minibatch loss at step 1500: 0.649688
Minibatch accuracy: 90.6%
Validation accuracy: 86.8%

Minibatch loss at step 2000: 0.903201
Minibatch accuracy: 84.4%
Validation accuracy: 86.7%

Minibatch loss at step 2500: 1.005441
Minibatch accuracy: 81.2%
Validation accuracy: 87.4%

Minibatch loss at step 3000: 0.889266
Minibatch accuracy: 85.2%
Validation accuracy: 88.0%

Test accuracy when beta is 0.000127: 93.6%
Test accuracy when beta is 0.000127: 93.6%


[None]

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [24]:
batch_size = 128

hidden_nodes = 128
graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    beta_para = tf.placeholder(tf.float32)
    tf_keep_prob = tf.placeholder(tf.float32)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes], stddev=0.1))
    biases = tf.Variable(tf.zeros([hidden_nodes]))
    
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    relu_result = tf.nn.relu(logits)
    relu_result = tf.nn.dropout(relu_result, keep_prob=tf_keep_prob)
    
    valid_logits = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
    test_logits = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)

    weights_list = []
    bias_list = []
    hidden_data = relu_result
    l2_reg = 0
    
    for layer_index in range(5):
        weights_list.append(tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], stddev=0.1)))
        bias_list.append(tf.Variable(tf.zeros([hidden_nodes])))
        
        logits_hidden = tf.matmul(hidden_data, weights_list[layer_index]) + bias_list[layer_index]
        hidden_data = tf.nn.dropout(tf.nn.relu(logits_hidden), tf_keep_prob)
        
        l2_reg += tf.nn.l2_loss(weights_list[layer_index])
        
        valid_logits = tf.nn.relu(tf.matmul(valid_logits, weights_list[layer_index]) + bias_list[layer_index])
        test_logits = tf.nn.relu(tf.matmul(test_logits, weights_list[layer_index]) + bias_list[layer_index])


    #the last layer
    weights_hiden1 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels], stddev=0.1))
    biases_hiden1 = tf.Variable(tf.zeros([num_labels]))
    logits = tf.matmul(hidden_data, weights_hiden1) + biases_hiden1
    
    valid_logits = tf.matmul(valid_logits, weights_hiden1) + biases_hiden1
    test_logits = tf.matmul(test_logits, weights_hiden1) + biases_hiden1

    l2_reg = l2_reg + tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights_hiden1)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta_para*l2_reg

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(valid_logits)
    test_prediction = tf.nn.softmax(test_logits)

In [25]:
num_steps = 3001
# beta_value = 0.000127
beta_value = 0.001
results = []

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_para: beta_value, tf_keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("")
        
  session.run(test_prediction, feed_dict={tf_keep_prob: 1})
  message = "Test accuracy when beta is %f: %.1f%%" % (beta_value, accuracy(test_prediction.eval(), test_labels))
  print(message)
  results.append(message)

[print(message) for message in results]    

Initialized
Minibatch loss at step 0: 3.239804
Minibatch accuracy: 11.7%
Validation accuracy: 10.2%

Minibatch loss at step 500: 1.755270
Minibatch accuracy: 51.6%
Validation accuracy: 56.3%

Minibatch loss at step 1000: 1.409796
Minibatch accuracy: 57.0%
Validation accuracy: 59.0%

Minibatch loss at step 1500: 1.157352
Minibatch accuracy: 64.8%
Validation accuracy: 71.4%

Minibatch loss at step 2000: 1.051406
Minibatch accuracy: 74.2%
Validation accuracy: 74.5%

Minibatch loss at step 2500: 1.133217
Minibatch accuracy: 74.2%
Validation accuracy: 74.3%

Minibatch loss at step 3000: 1.029745
Minibatch accuracy: 78.9%
Validation accuracy: 75.0%

Test accuracy when beta is 0.001000: 81.5%
Test accuracy when beta is 0.001000: 81.5%


[None]